In [1]:
import json
import os
import time

from datetime import datetime

os.getcwd()
 
from os.path import dirname, realpath
import sys
sys.path.append(dirname(dirname(realpath(os.getcwd()))))
import pickle
import torch
import multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from collections import Counter
from copy import deepcopy
import datetime
import tqdm
import numpy as np
import random
import pandas as pd

from collections import Counter
sys.path.append("./src/")

src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

def parse_date(date_str):
    if date_str == 'NA' or date_str is None:
        return datetime.datetime(9999,1,1,0,0)
    else:
        if len(date_str) == 10:
            format_str = '%Y-%m-%d'
        else:
            raise Exception("Format for {} not recognized!".format(date_str))
    return datetime.datetime.strptime(date_str, format_str)


VTE_path = 'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei'

In [2]:
race_bmi = pd.read_csv(os.path.join(VTE_path, 'samples/ad.csv'))

In [3]:
race_bmi['PatientICN'] =  race_bmi.PatientICN.astype(str)


In [7]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
m = testDF[["patient_id", "index_date","diag_date", "obs_time_end",  "outcome","outcome_date" ]]

In [8]:
m = testDF[["patient_id", "index_date","diag_date", "obs_time_end",  "outcome","outcome_date" ]]

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]
m1 = pd.concat([m_VE1, m_VE0, m0])
m1['PatICN'] =  m1.patient_id.astype(str)

m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
m_VE0['PatICN'] =  m_VE0.patient_id.astype(str)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_80112\1629420586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_80112\1629420586.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_80112\1629420586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a 

In [4]:
cbc_path = os.path.join(VTE_path, 'composite\\phe_CompositeLab.json')
Y = json.load(open(os.path.join(cbc_path), 'r'))

In [9]:
for p in Y.keys():
    Y[p]['Race'] = int(race_bmi [ race_bmi ['PatientICN']==p ]['race'].values[0]) 
    Y[p]['BMI'] = int(race_bmi [ race_bmi ['PatientICN']==p ]['bmi'].values[0]) 
    if Y[p]['gender'] == "M":
        Y[p]['gender'] = 1
    if Y[p]['gender'] == "F":
        Y[p]['gender'] = 0
    Y[p]['dxdate'] =  m1 [ m1 ['PatICN']==p ]['diag_date'].values[0]

    if sum( p  ==  m_VE0.PatICN ) ==1 :
        m_p = m_VE0[  p  ==  m_VE0.PatICN ] 
        Yp = Y[p]['events']
        m_date = m_p['model_date'].astype('datetime64[ns]')
        Yp_f = filter(lambda x: parse_date(x['admdate']) < m_date.ravel()[0] , Yp)
        Yp_new = list( Yp_f)
        Y[p]['events']  = Yp_new 

In [10]:
new_path = os.path.join(VTE_path, 'composite\\New_Composite.json')

In [11]:
with open(new_path, 'w') as fwd:
    print("Saving to {}\n".format(new_path))
    json.dump(Y, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\composite\New_Composite.json



In [ ]:
Y = 0

In [16]:
import gc

In [17]:

def train_dev_test_file_random_split(metafile, train_size=0.8, dev_size=0.1, test_size=0.1):
    '''
    This function randomly splits a json file into train, dev and test datasets, and 
    automatically saved splitted files under train, dev and test folders.
    Note: there is special handling for non-string type diagnosis codes when necessary
    '''
    par_dir = os.path.dirname(metafile)
    train_dir = os.path.join(par_dir, 'train')
    dev_dir = os.path.join(par_dir, 'dev')
    test_dir = os.path.join(par_dir, 'test')
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)
    if not os.path.exists(dev_dir):
        os.mkdir(dev_dir)
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    
    print('Loading data: {}'.format(metafile))
    dat = json.load(open(metafile, 'r'))

    # change phecodes datatype to string
    for p in dat.keys():
        for i in range(len(dat[p]['events'])):
            dat[p]['events'][i]['codes'] = str(dat[p]['events'][i]['codes'])

    all = [p for p in dat.keys()]
    random.shuffle(all)
    dev_ind_start = int(len(all) * train_size)
    test_ind_start = int(len(all) * (train_size + dev_size))

    # Train sampling
    train_sample = all[:dev_ind_start]
    train = {p: dat[p] for p in train_sample}
    for p in train:
        train[p]['split_group'] = 'train'
    train_path = os.path.join(train_dir, 'train.json')
    with open(train_path, 'w') as fwt:
        print("Saving to {}\n".format(train_path))
        json.dump(train, fwt, indent=None)
    del train
    gc.collect()

    # Dev sampling
    dev_sample = all[dev_ind_start:test_ind_start]
    dev = {p: dat[p] for p in dev_sample}
    for p in dev:
        dev[p]['split_group'] = 'dev'
    dev_path = os.path.join(dev_dir, 'dev.json')
    with open(dev_path, 'w') as fwd:
        print("Saving to {}\n".format(dev_path))
        json.dump(dev, fwd, indent=None)
    del dev
    gc.collect()

    # Test sampling
    test_sample = all[test_ind_start:]
    test = {p: dat[p] for p in test_sample}
    for p in test:
        test[p]['split_group'] = 'test'

    test_path = os.path.join(test_dir, 'test.json')
    with open(test_path, 'w') as fwtt:
        print("Saving to {}\n".format(test_path))
        json.dump(test, fwtt, indent=None)
    del test
    gc.collect()

In [18]:
train_dev_test_file_random_split(new_path, train_size=0.6, dev_size=0.2, test_size=0.2)


Loading data: Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\composite\New_Composite.json
Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\composite\train\train.json

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\composite\dev\dev.json

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\composite\test\test.json



In [1]:
Y = 0

In [2]:
lis = [1,1,1,2,2,2,3,3,3]

In [4]:
lis[7:5]

[]

In [5]:
lis[2:5]

[1, 2, 2]